## Demo notebook for fitting the model of _Brunton et. al. (2013)_

This is some boilerplate, for loading packages necessary for running the notebook

In [1]:
using Distributed
addprocs(2);

In [2]:
using pulse_input_DDM

┌ Info: Recompiling stale cache file /usr/people/briandd/.julia/compiled/v1.0/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1190


      From worker 3:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/2xipw/src/ImageFiltering.jl:24.
      From worker 2:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/2xipw/src/ImageFiltering.jl:24.


Define dictionaries with some generative parameters, a Boolean vector of which ones to fit, and initial values.

In [3]:
#parameters of the latent model
pz_gen = Dict("generative" => vcat(1.,13.,-0.5,10.,1.,0.4,0.02), 
    "name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

#parameters for the choice observation
pd_gen = Dict("generative" => vcat(0.1,0.1), 
    "name" => vcat("bias","lapse"), "fit" => trues(2), 
    "initial" => vcat(0.,0.5));

## simulate choices from a model, to check that generative parameters can be found

In [4]:
#define how many trials to generate data from and the time binning scale (in seconds)
ntrials,dt = Int(0.1e4), 1e-2

(1000, 0.01)

In [5]:
#generate some simulated clicks times and trial durations
data = pulse_input_DDM.sample_clicks(ntrials,dt);

In [6]:
#simulate choices from the model, given the generative parameters
pulse_input_DDM.sampled_dataset!(data, pz_gen["generative"], pd_gen["generative"]; num_reps=1, rng=4);

In [7]:
#compute the likelihood of the data, given the generative parameters
compute_LL(pz_gen["generative"], pd_gen["generative"],data)

-402.06250680341043

In [ ]:
#find the ML parameters with gradient descent
@time pz_gen, pd_gen, = optimize_model(pz_gen, pd_gen, data)

┌ Warning: some parameter(s) at lower bound. bumped it (them) up 1/4 from the lower bound.
└ @ pulse_input_DDM /mnt/bucket/people/briandd/Projects/pulse_input_DDM.jl/src/wrapper_functions.jl:67


Iter     Function value   Gradient norm 
     0     4.928052e+02     1.508580e+02
     1     4.076504e+02     1.456861e+01
     2     4.045523e+02     1.454315e+01
     3     4.023401e+02     6.905086e+00
     4     3.991265e+02     4.025686e+00
     5     3.986615e+02     3.673119e+00
     6     3.984288e+02     5.499980e+00
     7     3.983185e+02     3.437827e+00
     8     3.982588e+02     1.381193e+00
     9     3.978124e+02     5.395541e+00
    10     3.976176e+02     4.885572e+00
    11     3.971240e+02     2.923723e+00
    12     3.968307e+02     1.792665e+00
    13     3.966490e+02     5.467875e+00


In [ ]:
#compute the Hessian of the LL landscape, to compute confidence intervals on the parameters
compute_H_CI(pz_gen, pd_gen, data, dt)

In [ ]:
#identify which ML parameters have generative parameters within the CI 
pz_gen["within_bounds"] = (pz_gen["CI_minus"] .< pz_gen["generative"]) .& 
    (pz_gen["CI_plus"] .> pz_gen["generative"])
pd_gen["within_bounds"] = (pd_gen["CI_minus"] .< pd_gen["generative"]) .& 
    (pd_gen["CI_plus"] .> pd_gen["generative"]);

In [ ]:
using DataFrames
DataFrames.DataFrame(pz_gen)

In [ ]:
DataFrames.DataFrame(pd_gen)

In [ ]:
#sample data from the model using the ML parameters

#simulate choices from the model, given the generative parameters
sampled_data = deepcopy(data)
pulse_input_DDM.sampled_dataset!(sampled_data, pz_gen["final"], pd_gen["final"]; num_reps=1, rng=4);

#compute the final click difference, which will dictate the correct choice
ΔLR = map((nT,L,R)->diffLR(nT,L,R,dt),data["nT"],data["leftbups"],data["rightbups"])
ΔLR = map(x-> x[end], ΔLR);

In [ ]:
#bin the real choices and the sampled choices
using Pandas
nbins = 15;
conds,qcut_bins = qcut(ΔLR,nbins,labels=false,retbins=true);
conds = conds .+ 1;

frac_choice_sampled = fill!(Vector{Float64}(undef,nbins),NaN)
frac_choice = fill!(Vector{Float64}(undef,nbins),NaN)
for i = 1:nbins
    frac_choice_sampled[i] = mean(sampled_data["pokedR"][conds .== i])
    frac_choice[i] = mean(data["pokedR"][conds .== i])
end

In [ ]:
#fit a GLM to the real choices and the simulated choices
using GLM
GLM_dataframe = DataFrames.DataFrame(X= ΔLR, Y = data["pokedR"])
GLM_data = glm(@formula(Y ~ X), GLM_dataframe, Binomial(), LogitLink())
GLM_samplesframe = DataFrames.DataFrame(X= ΔLR, Y = sampled_data["pokedR"])
GLM_samples = glm(@formula(Y ~ X), GLM_samplesframe, Binomial(), LogitLink())

In [ ]:
using PyPlot
fig = figure(figsize=(6,6))
ax = subplot(111)

scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice, color="red", label="data")
scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_sampled, color="grey", label="sampled")

PyPlot.plot(sort(GLM_dataframe[:X]), sort(predict(GLM_data)), color="red")
PyPlot.plot(sort(GLM_samplesframe[:X]), sort(predict(GLM_samples)), color="grey")

ylabel("% poked R")
xlabel(L"\Delta{LR}")
ax[:spines]["top"][:set_color]("none") # Remove the top axis boundary
ax[:spines]["right"][:set_color]("none") # Remove the top axis boundary
legend()

## fit real data

In [ ]:
path = "/mnt/bucket/labs/brody/vdtang/P055_processed"
ratnames=["P055"]
sessids=[[17061800,18041700,16052600,16070500,16111700,18010300,18022600,17122200,
        17081200,17092600,17051900,18030500,17011800,18012200,17011500,17062500,
        17112700,18032700,16082200,18060800,18032500,17082900,17080500,17081000,
        17052300,17090800,17090900,18041500,18040500,17122000]];

In [ ]:
data = aggregate_choice_data(path, sessids, ratnames, dt)

## saving things

In [ ]:
using JLD2
JLD2.@save "/usr/people/briandd/Projects/pulse_input_DDM.jl/data/results/marino.jld" pz pd pzF pdF